In [8]:
import csv
import spacy
import nltk
import pprint

nlp = spacy.load('en_core_web_sm')

In [7]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm


    Linking successful
    /usr/local/lib/python3.7/site-packages/en_core_web_sm -->
    /Users/erdmann/Library/Python/3.7/lib/python/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [106]:
#Eingabe: Eine Zeile der Liste mit den Metadaten, so wie in der Beispieldatei

def get_ConflictsOfInterests(metadata):
    #angenommen, der Conflict-of-interest parameter steht an der dritten Stelle, sonst hier anpassen
    #hier wird der Parsetree mit Spacy gebaut
    doc = nlp(metadata[3])
    candidates=[]
    
    #geht jeden Satz (und dabei auch jedes Verb) durch
    for token in doc:
        
        #Wurzel (verb) finden
        if token.head.text == token.text and token.pos_ == 'VERB':
            
            
            #Autor wird gefunden, glücklicherweise sind die Sätze fast alle so strukturiert, dass der gesponserte links steht
            left_side = [' '.join([t.text for t in token.subtree]) for token in token.lefts if token.pos_ in ['PROPN','NOUN','PRON']]
            
            #wenn kein neues Subjekt da ist (oder er/sie), behalten wir einfach das vorherige
            if len(left_side)==1 and left_side[0].lower() not in ['he','she']:
                candidate_left = left_side[0]
                    
        
                
            #findet die Relation (was supported, was funded etc), eigentlich unnötig for now, deswegen auskommentiert       
            #verbterm = ' '.join([' '.join([t.text for t in token.subtree]) for token in token.lefts if not token.pos_ in ['PROPN','NOUN','PRON']])+' '+token.text
                
            
            #findet die sponsornenden Firmen
            #weil nicht immer eindeutig nehmen wir erstmal alle möglichen Teilstrings hier auf, das kann man vllt hinterher noch schöner machen aber für die Suche ist dann zumindest alles da drin
            right_side = [' '.join([t.text for t in token.subtree]) for token in token.rights]
            candidates_right = []
                
            #fängt an mit allen Wurzeln rechts des Verbs und iteriert dann durch
            children = [c for c in token.rights]
            while children!=[]:
                c = children.pop()
                    
                #wenn es ein Eigenname ist wird der Substring aufgenommen
                if c.pos_=='PROPN':
                    candidates_right.append(' '.join([t.text for t in c.subtree]))
                        
                #in jedem Fall werden alle Kinder weiter verarbeitet
                children=children+[d for d in c.children]
                    
            #die Kandidatenliste für mögliche Interessenskonflikte wird dann erstellt
            candidates = candidates+[[candidate_left, c] for c in candidates_right]
            
            
            
            
        ########'TODOS'##############
        #die linken Seiten auflösen, so dass geschaut wird, ob es einer der Autoren aus der Autorenliste ist und wenn nicht für jeden Autor einen Interessenskonflikt erstellen
        #es werden noch viel zu viele Unternehmenskandidaten erstellt, vllt kann man das noch ein wenig verbessern  
            
            
            
    return(candidates)
            

In [112]:
with open('test_closures.csv','r') as file:
    reader = csv.reader(file)
    rows = [r for r in reader]
    
    conflicts = set([r[3] for r in rows])
    
    
    filtered = [r for r in rows if r[3]!="The authors declare no conflict of interest." and r[3]!='' and r[3]!="None."]
    for s in filtered:
        for c in get_ConflictsOfInterests(s):
            print(c)
        print()
        


['The ALFSG', 'the National Institutes of Health ( National Institute of Diabetes , Digestive and Kidney Disease )']
['The ALFSG', 'National Institute of Diabetes , Digestive and Kidney Disease']
['The ALFSG', 'Digestive and Kidney Disease']
['The ALFSG', 'Kidney Disease']
['The ALFSG', 'Kidney']
['The ALFSG', 'Diabetes']
['The ALFSG', 'National']
['The ALFSG', 'Health']
['The ALFSG', 'National']

['The ALFSG', 'the National Institutes of Health ( National Institute of Diabetes , Digestive and Kidney Disease )']
['The ALFSG', 'National Institute of Diabetes , Digestive and Kidney Disease']
['The ALFSG', 'Digestive and Kidney Disease']
['The ALFSG', 'Kidney Disease']
['The ALFSG', 'Kidney']
['The ALFSG', 'Diabetes']
['The ALFSG', 'National']
['The ALFSG', 'Health']
['The ALFSG', 'National']

['The ALFSG', 'the National Institutes of Health ( National Institute of Diabetes , Digestive and Kidney Disease )']
['The ALFSG', 'National Institute of Diabetes , Digestive and Kidney Disease']
[

['Laura E. Targownik', 'Takeda Canada , Janssen Canada , Pfizer Canada , and Mallinkrodt USA']
['Laura E. Targownik', 'Janssen Canada , Pfizer Canada , and Mallinkrodt USA']
['Laura E. Targownik', 'Pfizer Canada , and Mallinkrodt USA']
['Laura E. Targownik', 'Mallinkrodt USA']
['Laura E. Targownik', 'Mallinkrodt']
['Laura E. Targownik', 'Pfizer']
['Laura E. Targownik', 'Janssen']
['Laura E. Targownik', 'Takeda']
['Charles N. Bernstein', 'the Bi gham Chair in Gastroenterology']
['Charles N. Bernstein', 'Gastroenterology']
['Charles N. Bernstein', 'gham']
['Charles N. Bernstein', 'Bi']
['Charles N. Bernstein', 'Mylan Pharmaceuticals']
['Charles N. Bernstein', 'Mylan']
['Charles N. Bernstein', 'Abbvie Canada , Ferring Canada , Janssen Canada , Napo Pharmaceuticals , Pfizer Canada , Shire Canada , Takeda Canada']
['Charles N. Bernstein', 'Ferring Canada , Janssen Canada , Napo Pharmaceuticals , Pfizer Canada , Shire Canada , Takeda Canada']
['Charles N. Bernstein', 'Janssen Canada , Napo P

['This study', 'NGM Biopharmaceuticals , South San Francisco , CA']
['This study', 'CA']
['This study', 'South San Francisco ,']
['This study', 'San']
['This study', 'South']
['This study', 'NGM']
['The study drug , NGM282 ,', 'NGM Biopharmaceuticals']
['The study drug , NGM282 ,', 'NGM']
['M.C.', 'National Institutes of Health']
['M.C.', 'Health']
['M.C.', 'National']
['M.C.', 'R01-Disha Khemani92179']
['M.C.', 'R01-Disha']
['The study', 'the Mayo Clinic Clinical Research Unit , which is supported by grant UL1-TR002377 from NIH']
['The study', 'NIH']
['The study', 'Clinical Research']
['The study', 'Clinical']
['The study', 'Clinic']
['The study', 'Mayo']

['K.K. and S.J.R.', 'NGM Biopharmaceuticals']
['K.K. and S.J.R.', 'NGM']

['This study', 'NGM Biopharmaceuticals , South San Francisco , CA']
['This study', 'CA']
['This study', 'South San Francisco ,']
['This study', 'San']
['This study', 'South']
['This study', 'NGM']
['The study drug , NGM282 ,', 'NGM Biopharmaceuticals']
['The s

['Fondation ARC', 'la']
['Fondation ARC', 'de']
['Fondation ARC', 'National']
['Fondation ARC', 'Institut']
['Fondation ARC', 'et de la Recherche Médicale ( INSERM )']
['Fondation ARC', 'INSERM']
['Fondation ARC', 'Recherche']
['Fondation ARC', 'et de la']
['Fondation ARC', 'et']
['Fondation ARC', 'Santé']
['Fondation ARC', 'de']
['Fondation ARC', 'National']
['Fondation ARC', 'Institut']
['Fondation ARC', 'Région Ile - de - France ( CORDDIM )']
['Fondation ARC', 'Région Ile - de - France']
['Fondation ARC', 'de']
['Fondation ARC', 'Ile']
['Fondation ARC', 'Région']

['Fondation ARC', 'Institut National de la Santé et de la Recherche Médicale ( INSERM ) , Institut National de la Recherche Agronomique ( INRA ) , Conservatoire National des Arts et Métiers ( CNAM ) , and Université Paris 13 .']
['Fondation ARC', 'Institut National de la Recherche Agronomique ( INRA ) , Conservatoire National des Arts et Métiers ( CNAM ) , and Université Paris 13 .']
['Fondation ARC', 'Conservatoire Nation

['Fondation ARC', 'Institut National de la Santé et de la Recherche Médicale ( INSERM ) , Institut National de la Recherche Agronomique ( INRA ) , Conservatoire National des Arts et Métiers ( CNAM ) , and Université Paris 13 .']
['Fondation ARC', 'Institut National de la Recherche Agronomique ( INRA ) , Conservatoire National des Arts et Métiers ( CNAM ) , and Université Paris 13 .']
['Fondation ARC', 'Conservatoire National des Arts et Métiers ( CNAM ) , and Université Paris 13 .']
['Fondation ARC', 'Université Paris 13 .']
['Fondation ARC', 'Université']
['Fondation ARC', '( CNAM )']
['Fondation ARC', 'et Métiers']
['Fondation ARC', 'des']
['Fondation ARC', 'National']
['Fondation ARC', 'Conservatoire']
['Fondation ARC', 'INRA']
['Fondation ARC', 'Recherche']
['Fondation ARC', 'la']
['Fondation ARC', 'de']
['Fondation ARC', 'National']
['Fondation ARC', 'Institut']
['Fondation ARC', 'et de la Recherche Médicale ( INSERM )']
['Fondation ARC', 'INSERM']
['Fondation ARC', 'Recherche']
[

...

['This work', 'Felix Stickel and Thorsten Buch']
['This work', 'Thorsten Buch']
['This work', 'Thorsten']
['This work', 'Felix']
['This work', 'the Swiss National Funds ( SNF no . 310030_169196 ) and the Swiss Foundation for Alcohol Research ( SSA )']
['This work', 'the Swiss Foundation for Alcohol Research ( SSA )']
['This work', 'Alcohol Research ( SSA']
['This work', 'Research']
['This work', 'Alcohol']
['This work', 'Swiss']
['This work', 'SNF no . 310030_169196']
['This work', 'National']
['This work', 'Swiss']
['This project', 'the Liver Systems Medicine , the ERACoSysMed project DYNAFrank LammertOW and EPITrio through the German Ministry for Research and Education ( BmBF ) to Jochen Hampe']
['This project', 'Jochen Hampe']
['This project', 'Jochen']
['This project', 'the German Ministry for Research and Education ( BmBF']
['This project', 'Research and Education ( BmBF']
['This project', 'Research and Education']
['This project', 'Education']
['This project', 'DYNAFrank LammertO

['This work', 'Felix Stickel and Thorsten Buch']
['This work', 'Thorsten Buch']
['This work', 'Thorsten']
['This work', 'Felix']
['This work', 'the Swiss National Funds ( SNF no . 310030_169196 ) and the Swiss Foundation for Alcohol Research ( SSA )']
['This work', 'the Swiss Foundation for Alcohol Research ( SSA )']
['This work', 'Alcohol Research ( SSA']
['This work', 'Research']
['This work', 'Alcohol']
['This work', 'Swiss']
['This work', 'SNF no . 310030_169196']
['This work', 'National']
['This work', 'Swiss']
['This project', 'the Liver Systems Medicine , the ERACoSysMed project DYNAFrank LammertOW and EPITrio through the German Ministry for Research and Education ( BmBF ) to Jochen Hampe']
['This project', 'Jochen Hampe']
['This project', 'Jochen']
['This project', 'the German Ministry for Research and Education ( BmBF']
['This project', 'Research and Education ( BmBF']
['This project', 'Research and Education']
['This project', 'Education']
['This project', 'DYNAFrank LammertO


['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David Katzka092217']
['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David']
['Rena Yadlapati and John E. Pandolfino', 'R01']
['Rena Yadlapati and John E. Pandolfino', 'NIH']


['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David Katzka092217']
['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David']
['Rena Yadlapati and John E. Pandolfino', 'R01']
['Rena Yadlapati and John E. Pandolfino', 'NIH']


['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David Katzka092217']
['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David']
['Rena Yadlapati and John E. Pandolfino', 'R01']
['Rena Yadlapati and John E. Pandolfino', 'NIH']


['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David Katzka092217']
['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David']
['Rena Yadlapati and John E. Pandolfino', 'R01']
['Rena Yadlapati and John E. Pandolfino', 'NIH']


['Rena Yadlapati and John E. Pandolfino', 'NIH R01 David Katzka092217']
['R

['The NACSELD registry', 'Grifols Pharmaceuticals']
['The NACSELD registry', 'Grifols']

['Jasmohan S. Bajaj , Florence Wong , Guadalupe Garcia - TsaoT , Scott W. Biggins , JO , K. Rajender Reddy , and Puneeta Tandon', 'Grifols Pharmaceuticals']
['Jasmohan S. Bajaj , Florence Wong , Guadalupe Garcia - TsaoT , Scott W. Biggins , JO , K. Rajender Reddy , and Puneeta Tandon', 'Grifols']

['The NACSELD registry', 'Grifols Pharmaceuticals']
['The NACSELD registry', 'Grifols']

['Jasmohan S. Bajaj , Florence Wong , Guadalupe Garcia - TsaoT , Scott W. Biggins , JO , K. Rajender Reddy , and Puneeta Tandon', 'Grifols Pharmaceuticals']
['Jasmohan S. Bajaj , Florence Wong , Guadalupe Garcia - TsaoT , Scott W. Biggins , JO , K. Rajender Reddy , and Puneeta Tandon', 'Grifols']

['The NACSELD registry', 'Grifols Pharmaceuticals']
['The NACSELD registry', 'Grifols']

['Jasmohan S. Bajaj , Florence Wong , Guadalupe Garcia - TsaoT , Scott W. Biggins , JO , K. Rajender Reddy , and Puneeta Tandon', 'Grif

NameError: free variable 'candidate_left' referenced before assignment in enclosing scope